# DL model embedding (TFLEARN)

In [ ]:
!pip install tflearn

     |████████████████████████████████| 112kB 8.4MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127301 sha256=35e3d85777f428015c1c93f3afdb709631bf8c8ce5a292384b7604c4a0ea580e
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [ ]:
from tflearn.layers import fully_connected,embedding,dropout
from tflearn.layers import batch_normalization,bidirectional_rnn
from tflearn.layers.recurrent import GRUCell,BasicLSTMCell
from tflearn.layers import input_data
from tflearn.data_utils import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tflearn import regression,DNN

In [ ]:
vocabulary_size = 5000

one_hot_object = [one_hot(word,vocabulary_size) for word in X_data]
# one_hot_object


sent_length_ = 40
embedded_words = pad_sequences(
    one_hot_object,
    maxlen = sent_length_,
    padding="post"
)
embedded_words




from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(embedded_words,Y_data.reshape(-1,1),test_size=0.15)

In [ ]:
embedding_vectors = 40



network = input_data(shape=(None,sent_length_))

network = embedding(network,input_dim=vocabulary_size,output_dim=embedding_vectors)

network = bidirectional_rnn(network,rnncell_fw=BasicLSTMCell(180),rnncell_bw=BasicLSTMCell(180),return_seq=True,name="bidiectional_1_layer")
network = bidirectional_rnn(network,rnncell_fw=BasicLSTMCell(160),rnncell_bw=BasicLSTMCell(160),return_seq=False,name="bidiectional_2_layer")

network = fully_connected(network,n_units=140,name="Dense_1_layer",activation="sigmoid")
network = dropout(network,keep_prob=0.5)
network = fully_connected(network,n_units=120,name="Dense_2_layer",activation="sigmoid")
network = batch_normalization(network)


network = fully_connected(network,n_units=42,name="Dense_3_layer",activation="sigmoid")
network = dropout(network,keep_prob=0.5)

network = fully_connected(network,1,activation="sigmoid")
network = regression(network,optimizer="rmsprop",loss="binary_crossentropy",metric="accuracy",learning_rate=0.0005)


model = DNN(network)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell, unroll=True))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
model.fit(
    X_train,
    Y_train,
    validation_set=0.15,
    batch_size=16,
    n_epoch = 25,
    show_metric=True
)

Training Step: 72499  | total loss: 0.69315 | time: 136.442s
| RMSProp | epoch: 025 | loss: 0.69315 - binary_acc: 0.4696 -- iter: 46384/46392
Training Step: 72500  | total loss: 0.69315 | time: 143.052s
| RMSProp | epoch: 025 | loss: 0.69315 - binary_acc: 0.4539 | val_loss: 0.69315 - val_binary_acc: 0.4838 -- iter: 46392/46392
--


In [ ]:
model.evaluate(
    X_test,
    Y_test,
    batch_size=128
)

[0.4833887043189369]

In [ ]:
model.save("Sentiment_tflearn.h5")

INFO:tensorflow:/content/Sentiment_tflearn.h5 is not in all_model_checkpoint_paths. Manually adding it.


observation --> Tflearn API model has underperformed hence using either keras model or tfidf or BOW model